# Libraries

In [1]:
!pip install playwright

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 MB 16.9 MB/s eta 0:00:00


In [2]:
!playwright install

173.7 MiB [] 0% 0.0s173.7 MiB [] 0% 48.0s173.7 MiB [] 0% 32.6s173.7 MiB [] 0% 24.4s173.7 MiB [] 0% 16.0s173.7 MiB [] 0% 13.0s173.7 MiB [] 1% 9.6s173.7 MiB [] 1% 7.7s173.7 MiB [] 1% 6.8s173.7 MiB [] 2% 6.3s173.7 MiB [] 2% 5.6s173.7 MiB [] 3% 5.2s173.7 MiB [] 3% 4.8s173.7 MiB [] 4% 4.6s173.7 MiB [] 4% 4.5s173.7 MiB [] 5% 4.5s173.7 MiB [] 5% 5.0s173.7 MiB [] 5% 5.1s173.7 MiB [] 6% 5.1s173.7 MiB [] 6% 5.2s173.7 MiB [] 6% 5.3s173.7 MiB [] 6% 5.5s173.7 MiB [] 7% 5.5s173.7 MiB [] 7% 5.3s173.7 MiB [] 7% 5.4s173.7 MiB [] 7% 5.6s173.7 MiB [] 7% 5.7s173.7 MiB [] 8% 5.6s173.7 MiB [] 9% 5.3s173.7 MiB [] 9% 5.2s173.7 MiB [] 10% 4.9s173.7 MiB [] 10% 5.0s173.7 MiB [] 11% 4.8s173.7 MiB [] 12% 4.6s173.7 MiB [] 13% 4.4s173.7 MiB [] 13% 4.6s173.7 MiB [] 14% 4.5s173.7 MiB [] 14% 4.4s173.7 MiB [] 15% 4.2s173.7 MiB [] 16% 4.2s173.7 MiB [] 17% 4.1s173.7 MiB [] 17% 4.0s173.7 MiB [] 18% 3.9s173.7 MiB [] 18% 4.0s173.7 MiB [] 18% 3.9s173.7 MiB [] 19% 3.9s173.7 MiB [] 19% 4.0s173.7 MiB [] 20% 4.0s173.7 MiB [] 20% 

In [3]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
from transformers import pipeline
import os
import time
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import json
import pandas as pd
import spacy
from concurrent.futures import ThreadPoolExecutor, as_completed
import en_core_web_sm
from collections import Counter
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import re
import numpy as np
import asyncio
from playwright.async_api import async_playwright, TimeoutError
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
import nest_asyncio


# Scrapping Food Data

# Extracting categories links

In [ ]:
def get_links_from_page(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    links = []
    cards = soup.find_all('a', class_='mntl-link-list__link')
    for card in cards:
        link = card['href']
        links.append(link)
    return links

# URLs of the categories
urls = {
    "Meals": "https://www.allrecipes.com/recipes-a-z-6735880",
    "Ingredients": "https://www.allrecipes.com/ingredients-a-z-6740416",
    "Cuisines": "https://www.allrecipes.com/cuisine-a-z-6740455",
}

all_links = {}

for category, url in urls.items():
    links = get_links_from_page(url)
    all_links[category] = links
    print(f"{category} Links:")
    print(links)


Meals Links:
['https://www.allrecipes.com/recipes/23070/everyday-cooking/cookware-and-equipment/air-fryer/', 'https://www.allrecipes.com/recipes/16492/everyday-cooking/special-collections/allrecipes-allstars/', 'https://www.allrecipes.com/recipes/385/desserts/cakes/angel-food-cake/', 'https://www.allrecipes.com/recipes/102/appetizers-and-snacks/antipasto/', 'https://www.allrecipes.com/recipes/76/appetizers-and-snacks/', 'https://www.allrecipes.com/recipes/788/desserts/pies/apple-pie/', 'https://www.allrecipes.com/recipes/1333/side-dish/applesauce/', 'https://www.allrecipes.com/recipes/14913/appetizers-and-snacks/dips-and-spreads/artichoke-dip/', 'https://www.allrecipes.com/recipes/1537/bread/yeast-bread/bagels/', 'https://www.allrecipes.com/recipes/1673/side-dish/beans-and-peas/baked-beans/', 'https://www.allrecipes.com/recipes/343/bread/quick-bread/fruit-bread/banana-bread/', 'https://www.allrecipes.com/recipes/836/desserts/cookies/bar-cookies/', 'https://www.allrecipes.com/recipes/20

In [ ]:
all_links = [link for links in all_links.values() for link in links]

In [ ]:
len(all_links)

496

## Extracting Recipe Links

In [ ]:
def get_recipe_links_from_page(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    recipe_links = []

    # First section: class 'comp mntl-card-list-items mntl-document-card mntl-card card--image-top card card--no-image'
    recipe_anchors_1 = soup.find_all('a', class_='comp mntl-card-list-items mntl-document-card mntl-card card--image-top card card--no-image')
    for anchor in recipe_anchors_1:
        link = anchor.get('href')
        if link:
            recipe_links.append(link)

    # Second section: class 'comp mntl-card-list-items mntl-document-card mntl-card card card--no-image'
    recipe_anchors_2 = soup.find_all('a', class_='comp mntl-card-list-items mntl-document-card mntl-card card card--no-image')
    for anchor in recipe_anchors_2:
        link = anchor.get('href')
        if link:
            recipe_links.append(link)

    return recipe_links

# Extract recipe links from all provided URLs
all_recipe_links = []

for url in all_links:
    links = get_recipe_links_from_page(url)
    all_recipe_links.extend(links)
    print(f"Extracted {len(links)} recipe links from {url}")

Extracted 63 recipe links from https://www.allrecipes.com/recipes/23070/everyday-cooking/cookware-and-equipment/air-fryer/
Extracted 63 recipe links from https://www.allrecipes.com/recipes/16492/everyday-cooking/special-collections/allrecipes-allstars/
Extracted 18 recipe links from https://www.allrecipes.com/recipes/385/desserts/cakes/angel-food-cake/
Extracted 36 recipe links from https://www.allrecipes.com/recipes/102/appetizers-and-snacks/antipasto/
Extracted 63 recipe links from https://www.allrecipes.com/recipes/76/appetizers-and-snacks/
Extracted 63 recipe links from https://www.allrecipes.com/recipes/788/desserts/pies/apple-pie/
Extracted 33 recipe links from https://www.allrecipes.com/recipes/1333/side-dish/applesauce/
Extracted 54 recipe links from https://www.allrecipes.com/recipes/14913/appetizers-and-snacks/dips-and-spreads/artichoke-dip/
Extracted 26 recipe links from https://www.allrecipes.com/recipes/1537/bread/yeast-bread/bagels/
Extracted 63 recipe links from https://

In [ ]:
len(all_recipe_links)

25474

In [ ]:
unique_recipe_links = list(set(all_recipe_links))
len(unique_recipe_links)

17062

# Extracting recipes

In [ ]:
# Initialize a session
session = requests.Session()

# Function to extract recipe data from a given URL
def extract_recipe_data(url):
    response = session.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Extract Recipe Name
    recipe_name_elem = soup.find('div', class_='comp article-header--recipe mm-recipes-article-header mntl-article-header')
    if recipe_name_elem:
        recipe_name = recipe_name_elem.find('h1').text.strip()
    else:
        recipe_name = 'Not Found'  # Handle case where element is not found

    # Extract Ingredients
    ingredients_list = []
    ingredients_elem = soup.find('ul', class_='mm-recipes-structured-ingredients__list')
    if ingredients_elem:
        ingredients = ingredients_elem.find_all('li')
        for ingredient in ingredients:
            ingredients_list.append(ingredient.text.strip())
    else:
        ingredients_list = ['Not Found']  # Handle case where element is not found

    # Extract Directions
    directions_list = []
    directions_elem = soup.find('ol', class_='comp mntl-sc-block mntl-sc-block-startgroup mntl-sc-block-group--OL')
    if directions_elem:
        directions = directions_elem.find_all('li')
        for direction in directions:
            directions_list.append(direction.text.strip())
    else:
        directions_list = ['Not Found']  # Handle case where element is not found

    # Extract Total Time
    total_time_elem = soup.find('div', class_='mm-recipes-details__value')
    if total_time_elem:
        total_time = total_time_elem.text.strip()
    else:
        total_time = 'Not Found'  # Handle case where element is not found

    # Extract Rating (if available)
    rating_elem = soup.find('div', class_='comp mm-recipes-review-bar__rating mntl-text-block type--squirrel-bold')
    if rating_elem:
        rating = rating_elem.text.strip()
    else:
        rating = 'Not Found'  # Handle case where rating element is not found

    # Extract Number of Ratings (if available)
    rating_count_elem = soup.find('div', class_='comp mm-recipes-review-bar__rating-count mntl-text-block type--squirrel')
    if rating_count_elem:
        rating_count = rating_count_elem.text.strip().strip('()')  # Remove surrounding parentheses
    else:
        rating_count = 'Not Found'  # Handle case where rating count element is not found

    # Return data as dictionary including the URL
    return {
        'Recipe Name': recipe_name,
        'Ingredients': ingredients_list,
        'Directions': directions_list,
        'Total Time': total_time,
        'Rating': rating,
        'Number of Ratings': rating_count,
        'URL': url
    }

# Function to process each recipe link
def process_recipe(link):
    try:
        recipe_data = extract_recipe_data(link)
        return recipe_data
    except Exception as e:
        print(f"Error processing {link}: {e}")
        return None

# Example unique_recipe_links list (should be replaced with actual links)

recipe_data_list = []

# Use ThreadPoolExecutor to process recipe links concurrently
with ThreadPoolExecutor(max_workers=10) as executor:  # Adjust max_workers as needed
    futures = [executor.submit(process_recipe, link) for link in unique_recipe_links]
    for future in tqdm(as_completed(futures), total=len(futures), desc='Processing Recipes'):
        recipe_data = future.result()
        if recipe_data:
            recipe_data_list.append(recipe_data)

# Create a DataFrame from the collected data
df = pd.DataFrame(recipe_data_list)
df.to_csv('recipes_data.csv', index=False)

Processing Recipes:   8%|▊         | 1413/17062 [05:21<1:55:13,  2.26it/s]

Error processing https://www.allrecipes.com/longform/chicken-wing-champions/: ("Connection broken: InvalidChunkLength(got length b'', 0 bytes read)", InvalidChunkLength(got length b'', 0 bytes read))


Processing Recipes:  31%|███▏      | 5356/17062 [20:38<28:07,  6.94it/s]

Error processing https://www.allrecipes.com/recipe/44975/easy-pizza-sauce-iii/: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


Processing Recipes:  43%|████▎     | 7351/17062 [28:37<35:10,  4.60it/s]

...

Processing Recipes:  51%|█████     | 8619/17062 [33:52<27:23,  5.14it/s]

Error processing https://www.allrecipes.com/recipe/203223/haydari-turkish-yogurt-dip/: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


Processing Recipes:  78%|███████▊  | 13303/17062 [54:13<15:16,  4.10it/s]

Error processing https://www.allrecipes.com/article/difference-between-granola-muesli/: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


Processing Recipes: 100%|██████████| 17062/17062 [1:12:03<00:00,  3.95it/s]


## Extracting Recipe Reviews

In [ ]:
# Apply the nest_asyncio patch
nest_asyncio.apply()

# Define the function to fetch reviews
async def fetch_reviews(url):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        await page.goto(url)

        try:
            # Wait for the reviews section to appear
            await page.wait_for_selector('.feedback__text', timeout=60000)  # Adjusted timeout to 60 seconds
        except TimeoutError:
            print(f"TimeoutError: Selector '.feedback__text' not found on {url}")
            await browser.close()
            return []  # Return empty list if no reviews are found

        # Extract reviews
        reviews_elements = await page.query_selector_all('.feedback__text')

        reviews = []
        if not reviews_elements:
            print(f"No review texts found on {url}")
        else:
            for review in reviews_elements[:10]:  # Limit to 10 reviews
                review_text = await review.inner_text()
                reviews.append(review_text.strip())

        await browser.close()
        return reviews

# Function to process each recipe and extract reviews
async def process_recipes(recipes_df):
    review_data = []
    for _, row in recipes_df.iterrows():
        url = row['URL']
        recipe_name = row['Recipe Name']
        reviews = await fetch_reviews(url)
        if reviews:  # Only add if reviews are not empty
            for review in reviews:
                review_data.append({'Recipe Name': recipe_name, 'Review': review})
        else:
            # Handle cases where no reviews are found
            review_data.append({'Recipe Name': recipe_name, 'Review': "No review texts found."})
    return review_data

# Convert the function to a synchronous call
def get_reviews_from_recipes(recipes_df):
    return asyncio.run(process_recipes(recipes_df))

# Extract reviews for the first 20 recipes
df_subset = df.head(20)
reviews_list = get_reviews_from_recipes(df_subset)

# Create a new DataFrame with the extracted reviews
reviews_df = pd.DataFrame(reviews_list)

TimeoutError: Selector '.feedback__text' not found on https://www.allrecipes.com/grilled-coconut-curry-tofu-kebabs-recipe-8645704
TimeoutError: Selector '.feedback__text' not found on https://www.allrecipes.com/article/what-can-you-eat-during-lent/


# Datasets

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/recipe_data.csv')
reviews_df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/reviews.xlsx')

In [10]:
reviews_df = reviews_df[reviews_df['Review'] != 'No review texts found.']
df = df[df['Recipe Name'] != 'Not Found']

In [11]:
df.shape

(16252, 6)

In [12]:
df.head(5)

,Recipe Name,Ingredients,Directions,Total Time,Rating,Number of Ratings
1,Sloppy Joes for a Crowd,"['3 tablespoons vegetable oil', '6 pounds grou...",['Heat 1 tablespoon oil in a large skillet ove...,20 mins,4.6,7
2,Crawfish-Stuffed Jalapenos,"['8 ounces bacon', '8 ounces bulk lean breakfa...","['Place bacon in a large, deep skillet. Cook o...",1 hr,4.7,3
3,"Breaded, Fried, Softly Spiced Tofu","['1 (16 ounce) package extra-firm tofu, draine...",['Cut pressed tofu into 1/2-inch thick slices;...,15 mins,4.3,280
4,Cabbage and Gnocchi,"['1 (16 ounce) package gnocchi', '¼ cup butter...",['Bring a large pot of lightly salted water to...,20 mins,4.6,7
5,Buttermilk Sausage Gravy,['1 (8 ounce) package pork link sausages (such...,['Place sausages and bacon drippings in a skil...,5 mins,2.8,5


# Recipe Classification

In [13]:
categories = {

    'main_course': ['chicken', 'beef', 'pork', 'steak', 'fish', 'shrimp', 'stir-fry', 'tenderloin', 'sausage',
                    'pasta', 'lasagna', 'mac', 'donair', 'gnocchi', 'ravioli', 'risotto', 'turkey', 'stuffed', 'paella', 'pasties',
                    'manicotti', 'spaghetti', 'linguine', 'meatballs', 'scallops', 'roast', 'ziti', 'arepas', 'rice', 'broil',
                    'matzo brei', 'quiche', 'tamales', 'gyro', 'noodle', 'tortilla', 'adobo', 'cordon bleu', 'brisket', 'chili',
                    'prime rib', 'catfish', 'lobster', 'swordfish', 'rabbit', 'veal chop', 'butternut squash', 'calzone',
                    'stroganoff', 'venison', 'gumbo', 'sosaties', 'chateaubriand', 'duck', 'squirrel', 'sabzi', 'salmon', 'nachos',
                    'ribs', 'sandwich', 'cornish hens', 'dinner', 'tenders', 'pinwheels', 'tofu', 'tetrazzini', 'sushi', 'burritos',
                    'hamburgers', 'tinga', 'fettuccine', 'pizza', 'tacos', 'pollo', 'burger', 'quesadillas', 'fajitas', 'hot dog','hotdog',
                    'tuna', 'kebab', 'wings', 'meatloaf', 'enchiladas', 'cioppino', 'kielbasa', 'coq', 'jambalaya', 'empanadas', 'lamb',
                    'boeuf', 'barbeque', 'tortellini', 'taco'],

    'soup': ['soup', 'bisque', 'broth', 'stew', 'chowder', 'gazpacho', 'minestrone', 'lentil', 'pho', 'borscht', 'gravy'],

    'dessert': ['cake', 'cookie', 'pudding', 'pie', 'crepe', 'brownie', 'tart', 'fudge', 'custard', 'chocolate', 'ice cream',
                'gingerbread', 'cobbler', 'crisp', 'galette', 'cornmeal', 'shortcake', 'bundt cake', 'cheesecake',
                'blondies', 'mousse', 'clafoutis', 'panna cotta', 'kuchen', 'cupcake', 'tiramisu', 'butterkuchen', 'meringue',
                'berry pavlovas', 'keufels', 'fruitcake', 'icing', 'flan', 'creme brulee', 'halwa', 'jelly', 'dessert', 'coffeecake',
                'caramelized', 'muffins', 'roll'],

    'drink': ['smoothie', 'punch', 'margarita', 'sangria', 'tea', 'cocktail', 'whiskey', 'slush', 'lemonade', 'gluehwein', 'martini',
              'milkshake', 'juice', 'coffee', 'liqueur', 'tequilla', 'milk', 'sangria'],

    'bread': ['bread', 'bun', 'dough', 'pita', 'ksra', 'damper', 'scone', 'baguette', 'cornbread', 'naan', 'focaccia', 'shortbread',
              'Hamburger Buns'],

    'salad': ['salad', 'tabbouleh', 'coleslaw', 'nicoise', 'insalata'],

    'side_dish': ['side', 'pilaf', 'salsa', 'marmalade', 'chutney', 'jam', 'fries', 'filling', 'cream', 'applesauce',
                  'cheese fingers', 'potatoes', 'stuffing', 'asparagus', 'fiddleheads', 'sauce', 'pickles',
                  'dressing', 'seasoning', 'sauce', 'guacamole', 'balsamic', 'pesto', 'syrup', 'hummus', 'falafel',
                  'tzatziki', 'dip', 'egg roll', 'patties', 'dumplings', 'spread', 'fritters', 'bites',
                  'rings', 'bruschetta', 'pot stickers', 'stuffed peppers', 'spring rolls', 'popper', 'crostata', 'pierogies',
                  'sambousa'],

    'breakfast': ['pancakes', 'oats', 'granola', 'porridge', 'waffles', 'pastry', 'strata', 'scones', 'oatmeal',
                  'croissants', 'cereal', 'french toast', 'breakfast', 'omelet', 'pudding', 'muesli'],

    'snack': ['popcorn', 'bars', 'seeds', 'balls', 'trail mix', 'nuts', 'crackers', 'granola bars', 'rice cakes', 'pretzels', 'chips',
              'jerky', 'blondies', 'cashews', 'biscotti', 'biscuits', 'snickerdoodles', 'churros', 'caramels']
}

In [14]:
def categorize_recipe_name(recipe_name):
    recipe_name = recipe_name.lower()

    # Define the priority order for categories
    priority_categories = ['salad', 'soup', 'bread', 'main_course', 'dessert', 'snack','breakfast', 'side_dish', 'drink'] # this is supposed to solve the issue with dishes like
    #rice cake, pizza dough, rice salad, chicken pie...

    # Initialize the best match variables
    best_match_category = 'other'
    highest_priority = len(priority_categories)  # Higher number means lower priority!

    # Check for the highest priority category based on keywords present
    for category in priority_categories:
        keywords = categories.get(category, [])
        # Check if any keyword is present in the recipe name
        if any(keyword in recipe_name for keyword in keywords):
            # Update best match if the current category has higher priority
            current_priority = priority_categories.index(category)
            if current_priority < highest_priority:
                best_match_category = category
                highest_priority = current_priority

    return best_match_category

In [15]:
df['Category'] = df['Recipe Name'].apply(categorize_recipe_name)

In [16]:
side_dish = df[df['Category'] == 'side_dish'][['Recipe Name']].rename(columns={'Recipe Name': 'side_dish'})
salad = df[df['Category'] == 'salad'][['Recipe Name']].rename(columns={'Recipe Name': 'salad'})
soup = df[df['Category'] == 'soup'][['Recipe Name']].rename(columns={'Recipe Name': 'soup'})
bread = df[df['Category'] == 'bread'][['Recipe Name']].rename(columns={'Recipe Name': 'bread'})
main_course = df[df['Category'] == 'main_course'][['Recipe Name']].rename(columns={'Recipe Name': 'main_course'})
dessert = df[df['Category'] == 'dessert'][['Recipe Name']].rename(columns={'Recipe Name': 'dessert'})
snack = df[df['Category'] == 'snack'][['Recipe Name']].rename(columns={'Recipe Name': 'snack'})
breakfast = df[df['Category'] == 'breakfast'][['Recipe Name']].rename(columns={'Recipe Name': 'breakfast'})
drink = df[df['Category'] == 'drink'][['Recipe Name']].rename(columns={'Recipe Name': 'drink'})

In [17]:
drink.shape

(356, 1)

In [18]:
other = df[df['Category'] == 'other']
other

,Recipe Name,Ingredients,Directions,Total Time,Rating,Number of Ratings,Category
1,Sloppy Joes for a Crowd,"['3 tablespoons vegetable oil', '6 pounds grou...",['Heat 1 tablespoon oil in a large skillet ove...,20 mins,4.6,7,other
12,Dulce de Leche,"['1 gallon milk', '1 vanilla bean', '4 ½ cups...","['In a large saucepan, bring milk to a boil ov...",15 mins,4.4,60,other
31,Baba Ghanoush,"['1 large eggplant', '2 to 4 tablespoons lemo...",['Preheat the oven to 400 degrees F (200 degre...,5 mins,4.4,467,other
33,Di's Delicious Deluxe Deviled Eggs,"['6 eggs', '½ stalk celery, finely chopped', ...",['Place eggs in a medium saucepan and cover wi...,20 mins,4.4,580,other
39,St. Patrick's Colcannon,"['3 pounds red potatoes, quartered', '1 head c...",['Place potatoes into a large saucepan and add...,25 mins,4.5,39,other
...,...,...,...,...,...,...,...
17001,Grilled Pattypan Squash With Garlic Butter,"['4 tablespoons unsalted butter, softened', '1...",['Preheat grill or grill pan to medium-high (4...,10 mins,Not Found,Not Found,other
17003,Panzanella Panini,"['1 French deli roll, split', '1 teaspoon bal...","['Preheat a skillet over medium-low heat.', 'S...",10 mins,4.7,124,other
17009,Air Fryer Kibbeh,"['1 cup medium-grind bulgur', '1 medium onion,...",['Soak bulgur in a bowl of water until soft to...,20 mins,Not Found,Not Found,other
17016,Air Fryer Waffle Egg in a Hole,"['1 frozen waffle', '1 large egg', 'salt and p...",['Preheat the air fryer to 350 degrees F (175 ...,2 mins,5.0,3,other


In [19]:
data = [
    ("side_dish", side_dish.shape[0]),
    ("salad", salad.shape[0]),
    ("soup", soup.shape[0]),
    ("bread", bread.shape[0]),
    ("main_course", main_course.shape[0]),
    ("dessert", dessert.shape[0]),
    ("snack", snack.shape[0]),
    ("breakfast", breakfast.shape[0]),
    ("drink", drink.shape[0]),
    ("other", other.shape[0])
]
df_shapes = pd.DataFrame(data, columns=['Category', 'Number of Recipes'])
df_shapes

,Category,Number of Recipes
0,side_dish,1122
1,salad,849
2,soup,1031
3,bread,896
4,main_course,5421
5,dessert,3250
6,snack,374
7,breakfast,382
8,drink,356
9,other,2571


In [20]:
side_dish = side_dish.reset_index(drop=True)
salad = salad.reset_index(drop=True)
soup = soup.reset_index(drop=True)
bread = bread.reset_index(drop=True)
main_course = main_course.reset_index(drop=True)
dessert = dessert.reset_index(drop=True)
snack = snack.reset_index(drop=True)
breakfast = breakfast.reset_index(drop=True)
drink = drink.reset_index(drop=True)

In [21]:
classified_recipes = pd.concat([side_dish, salad, soup, bread, main_course, dessert, snack, breakfast, drink], axis=1)

In [22]:
classified_recipes.head(20)

,side_dish,salad,soup,bread,main_course,dessert,snack,breakfast,drink
0,Salsa di Noci,Potato Salad with Beans and Yogurt Dressing,Buttermilk Sausage Gravy,"Breaded, Fried, Softly Spiced Tofu",Crawfish-Stuffed Jalapenos,Southwest-Style Egg Rolls,Ranch Oyster Crackers,Easy French Toast Waffles,Classic Spanish Sangria
1,Chinese-Style Baby Bok Choy with Mushroom Sauce,Greek Rice Salad,"Chard Lentil Soup, Lebanese-Style",Zucchini Walnut Bread,Cabbage and Gnocchi,Ricotta Cheese Cookies,Chef John's Lemon Bars,Amish Baked Oatmeal,Vietnamese Coffee
2,Tomatillo Salsa Verde,Avocado Pineapple Salad,Slow Cooker Lamb Stew,Crackling Cornbread,Sheet Pan Black Pepper Tofu and Broccoli,Caramelized Bananas,Energy Balls,Easy Strata and Variations,Pomegranate and Guava Smoothie
3,Olive Pecan Spread,Spicy Dorito® Taco Salad,Butternut Squash Soup with Cream Cheese,Banana Cornbread,Roasted Cherry Tomato Pasta,Honey Carrot Cake,Lemon Dream Bars,Ham and Cheese Omelet Casserole,Oat Milk
4,The Best Mashed Potatoes,Creamy Spiced Coleslaw,Cuban Black Bean Soup in the Slow Cooker,Norwegian Potato Flatbread (Lefse),The Best Lamb Chops,Spinach Cheddar Muffins,Mall Pretzels,Apple Oatmeal Breakfast Cups,Spinach and Banana Power Smoothie
5,Air Fryer Asparagus,Red Salad,Gluten-Free Sausage Gravy,No-Rise Pizza Dough,Italian Chicken with Garlic and Lemon,Cassava Flour-Apple Cake,Fluffy Whole Wheat Biscuits,Norwegian Waffles,POG Juice
6,Cherry Jam,Old Fashioned Pea Salad,Instant Pot Lebanese Lentil and Chard Soup,Cream Cheese Banana Bread,Perfect Crispy Fried Chicken,Bertha's Pecan Cream Pie,Oat-Free and Gluten-Free Granola Bars (Clean E...,Carne Asada Breakfast Burrito,Spiced Tea Mix
7,Cranberry Salsa,Black Bean and Corn Salad,Chili-Topped Potato Soup,Brown Sugar Shortbread Cookies,Chef John's Steak Pizzaiola,Mock Chocolate Cookie Crust,Honey Pumpkin Seeds,Blueberry Oatmeal,Never Bitter Lemonade
8,Healthier Double Tomato Bruschetta,Greek-Style Tuna Salad,Chicken with Barley Soup,Sticky Buns,Valencian Savory Empanadas,Boiled Fruit Cake,Sweet Pumpkin Seeds,Campfire Skillet Breakfast,Homemade Blackberry Lemonade
9,Cajun Baked French Fries,Spicy Tex-Mex Salad,Italian-Style Chicken Noodle Soup,Rye Pizza Dough,Salmon Patties with Dill Sauce,Fresh Peach Cake,Baked Pumpkin Seeds,Perfect Pumpkin Spice Waffles,Just-Right Pineapple Lemonade from Scratch


In [23]:
clean_df = df[df['Category'] != 'other']

In [24]:
clean_df.shape

(13681, 7)

# SVC

In [25]:
# Splitting the data
X_train, X_test, y_train, y_test, indices_train, indices_test = train_test_split(
    clean_df['Recipe Name'],
    clean_df['Category'],
    clean_df.index,
    test_size=0.2,
    random_state=42
)

# Vectorize text data
# Experiment with TF-IDF
vectorizer = TfidfVectorizer(stop_words='english')


X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

# Train and evaluate classifiers with different kernels
for kernel in ['linear', 'poly', 'rbf', 'sigmoid']:
    print(f"Evaluating SVC with {kernel} kernel")

    # Configure kernel-specific parameters
    if kernel == 'poly':
        classifier = SVC(kernel=kernel, degree=3)  # Set degree for polynomial kernel
    elif kernel == 'rbf':
        classifier = SVC(kernel=kernel, gamma='scale')  # Set gamma for RBF kernel
    else:
        classifier = SVC(kernel=kernel)  # No additional parameters needed for linear and sigmoid kernels

    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    print(f"Classification report for {kernel} kernel:\n{classification_report(y_test, y_pred)}")

Evaluating SVC with linear kernel
Classification report for linear kernel:
              precision    recall  f1-score   support

       bread       1.00      0.97      0.98       187
   breakfast       0.86      0.96      0.91        76
     dessert       0.98      0.98      0.98       645
       drink       0.97      0.88      0.92        64
 main_course       0.98      0.98      0.98      1103
       salad       1.00      0.99      1.00       163
   side_dish       0.95      0.98      0.96       217
       snack       0.87      0.91      0.89        74
        soup       1.00      1.00      1.00       208

    accuracy                           0.98      2737
   macro avg       0.96      0.96      0.96      2737
weighted avg       0.98      0.98      0.98      2737

Evaluating SVC with poly kernel
Classification report for poly kernel:
              precision    recall  f1-score   support

       bread       1.00      0.82      0.90       187
   breakfast       0.84      0.55      0

## Random Search

In [26]:
# Step 1: Splitting the Data
X_train, X_test, y_train, y_test = train_test_split(
    clean_df['Recipe Name'],
    clean_df['Category'],
    test_size=0.2,
    random_state=42
)

In [27]:
# Step 2: Vectorize Text Data
vectorizer = TfidfVectorizer(stop_words='english')
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

In [28]:
param_dist = {
    'C': [0.1, 1, 10],  # Narrower range of C values
    'kernel': ['linear', 'rbf'],  # Focus on linear and rbf kernels
    'gamma': ['scale'],  # Simplified for 'rbf'
}

In [29]:
# Step 4: Initialize SVC and RandomizedSearchCV
svc = SVC()

In [30]:
# Configure RandomizedSearchCV with parallel processing (n_jobs=-1 to use all cores)
random_search = RandomizedSearchCV(estimator=svc, param_distributions=param_dist,
                                   n_iter=6, cv=3, scoring='accuracy',
                                   random_state=42, n_jobs=-1)

In [31]:
# Step 5: Fit RandomizedSearchCV
random_search.fit(X_train_vec, y_train)

RandomizedSearchCV(cv=3, estimator=SVC(), n_iter=6, n_jobs=-1,
                   param_distributions={'C': [0.1, 1, 10], 'gamma': ['scale'],
                                        'kernel': ['linear', 'rbf']},
                   random_state=42, scoring='accuracy')

In [32]:
# Step 6: Output Best Parameters and Performance
print(f"Best parameters: {random_search.best_params_}")
print(f"Best cross-validation score: {random_search.best_score_}")

Best parameters: {'kernel': 'linear', 'gamma': 'scale', 'C': 10}
Best cross-validation score: 0.9788925438596491


In [33]:
# Step 7: Evaluate the Best Model on the Test Set
best_model = random_search.best_estimator_
y_pred = best_model.predict(X_test_vec)
print(f"Classification report for the best model:\n{classification_report(y_test, y_pred)}")

Classification report for the best model:
              precision    recall  f1-score   support

       bread       1.00      0.98      0.99       187
   breakfast       0.95      0.97      0.96        76
     dessert       0.99      0.98      0.99       645
       drink       0.95      0.97      0.96        64
 main_course       0.99      0.99      0.99      1103
       salad       1.00      0.99      1.00       163
   side_dish       0.94      0.98      0.96       217
       snack       0.91      0.93      0.92        74
        soup       1.00      1.00      1.00       208

    accuracy                           0.98      2737
   macro avg       0.97      0.98      0.97      2737
weighted avg       0.98      0.98      0.98      2737



# Sentiment Analysis

In [35]:
vader_file = '/content/drive/MyDrive/Colab Notebooks/vader_lexicon.txt'
vader_lex = pd.read_csv(vader_file, delimiter='\t', header=None, names=['Token', 'Score', 'STANDARD-DEVIATION', 'RAW-HUMAN-SENTIMENT-RATINGS'])

In [36]:
vader_lex = vader_lex[['Token', 'Score']]

## Adding newfood-related terms and their sentiment scores to the vader lexicon

In [37]:
new_words = {
    'tasty': 2.5,
    'flavorful': 2.0,
    'savory': 2.0,
    'authentic': 2.0,
    'stale': -2.0,
    'tasteless': -2.5,
    'greasy': -0.6,
    'blah': -2.5,
    'dusty': -2.0
}

new_words_df = pd.DataFrame(list(new_words.items()), columns=['Token', 'Score'])
vader_lex = pd.concat([vader_lex, new_words_df], ignore_index=True)

# VADER Sentiment Classifier

In [38]:
class VaderSentiment:
    def __init__(self, lexicon):
        self.lexicon_df = lexicon
        self.vader_dict = dict(zip(lexicon['Token'], lexicon['Score']))
        self.intensifiers = {'very': 0.5, 'extremely': 0.7, 'absolutely': 0.6}
        self.negations = {'not': -0.5, 'never': -0.6}

    def tokenize(self, text):
        return re.findall(r"[\w']+|[.,!?;]", text.lower())

    def adjust_score(self, score, text):
        if '!' in text:
            score += 0.292
        if '?' in text:
            score -= 0.18
        if any(char.isupper() for char in text):
            score *= 1.1
        return np.clip(score, -1, 1)

    def calculate_sentiment(self, text):
        tokens = self.tokenize(text)
        sentiment_score = 0

        for i, token in enumerate(tokens):
            if token in self.vader_dict:
                sentiment_score += self.vader_dict[token]

                # Apply intensity adjustments
                if i > 0 and tokens[i - 1] in self.intensifiers:
                    sentiment_score += self.intensifiers[tokens[i - 1]] * self.vader_dict[token]
                if i > 0 and tokens[i - 1] in self.negations:
                    sentiment_score -= self.vader_dict[token] * self.negations[tokens[i - 1]]

        sentiment_score = self.adjust_score(sentiment_score, text)
        return sentiment_score

    def get_sentiment_label(self, score):
        if score >= 0.05:
            return 'positive'
        elif score <= -0.05:
            return 'negative'
        else:
            return 'neutral'

    def analyze(self, reviews):
        results = []
        for review in reviews:
            score = self.calculate_sentiment(review)
            sentiment = self.get_sentiment_label(score)
            results.append((review, score, sentiment))
        return results

## Application to Food Reviews

In [39]:
vader = VaderSentiment(vader_lex)

In [40]:
reviews_df['Total Score'] = reviews_df['Review'].apply(vader.calculate_sentiment)
reviews_df['Sentiment'] = reviews_df['Total Score'].apply(vader.get_sentiment_label)

In [41]:
reviews_df

,Unnamed: 0,Recipe Name,Review,Total Score,Sentiment
0,0,Tuna Mushroom Casserole,This recipe worked out really well after some ...,1.00,positive
1,1,Tuna Mushroom Casserole,This makes a food batch for a simple recipe! R...,1.00,positive
2,2,Tuna Mushroom Casserole,It could use more taste. I didn't measure exac...,1.00,positive
3,3,Tuna Mushroom Casserole,I modified this recipe slightly. I used fresh ...,1.00,positive
4,4,Tuna Mushroom Casserole,I added a little Italian Seasoning and used It...,1.00,positive
...,...,...,...,...,...
151,151,Easy North Carolina Barbeque,Every time I make this for an event it gets ra...,1.00,positive
152,152,Tuna Avocado Salad,"Fast, easy and really delicious! I've had it o...",1.00,positive
153,153,Tuna Avocado Salad,I didn't find it terribly flavorful. I added s...,-0.66,negative
154,154,Tuna Avocado Salad,Very easy and tasty. Felt healthy eating this.,1.00,positive


In [42]:
reviews_df['Sentiment'].value_counts()

,count
Sentiment,
positive,142
negative,7
neutral,4


In [43]:
pd.set_option('display.max_colwidth', None)

In [44]:
reviews_df[reviews_df['Sentiment'] == 'negative'][['Review']]

,Review
23,"Awful...please don’t waste your good butter or almond flour on this recipe. The cornstarch overwhelms the recipe so badly that the cookie leaves a starch film in your mouth. I tried adding almond extract, like some suggested, no change."
26,"Blah. I had high hopes for this recipe because of the almond meal, but I think the cornstarch gives them an almost dusty mouth-feel. I wouldn't waste your time with this; traditional shortbread is much better and simpler."
59,No changes made but I may try a different recipe later
92,Boring. No flavor other than shrimp flavor and it’s not creamy at all. Just melted butter. It needs cheese and less butter and maybe some actual spices. Total miss
112,"This is a fairly bland pot roast recipe. Nothing bad, but our regular pot roast is much better."
138,"I was so disappointed with this recipe. I found as well that the sugar and oil seperated and when I finally got the sugar to melt (into a caramel-ish consistency) you have to work really fast before it hardens. I had to keep microwaving the pumpkin seeds to melt the caramel in order to coat all of the seeds. There is potential with this recipe, but I would definatly start off with a small batch of seeds, rather than use all that you have and waste them."
153,I didn't find it terribly flavorful. I added some radish for crunch and also some green onions. I don't think I'll make it again.


In [45]:
reviews_df[reviews_df['Sentiment'] == 'neutral'][['Review']]

,Review
54,It was gud
69,5 stars
77,I added about 1/4 cup of shredded coconut also I used chunk pineapple which I then sliced.
113,Cooking now.


In [46]:
# wrongly classified as positive
reviews_df['Review'].iloc[43]

'The Thyme was unfortunately the first flavor I and my spouse detected. I think it would be better without it. I ended up dusting them with more parm cheese and it then took on almost a pizza like flavor and smell.'

In [47]:
# wrongly classified as positive
reviews_df['Review'].iloc[82]

'I made them a little bigger and they came out more flat, dropping the dough on parchment paper. I melted 1/2 cup dark chocolate chips and 1 tablespoon butter and separately 1/2 cup milk chocolate chips and 1 tablespoon butter in the microwave, stirring well, then dipped half the macaroons in the dark chocolate mixture and half in the milk chocolate mixture, dipping the bottom and using a spoon to sprinkle some chocolate on top of the cookies. I laid the dipped cookies back on the parchment paper to cool and let the chocolate set.'

In [48]:
# wrongly classified as positive
reviews_df['Review'].iloc[144]

'Everything was going fine until it came time to add the sauce. There was so much of it that it was the wettest pulled pork I have ever seen. As others mentioned, way too heavy on the vinegar as well. Half the amount of sauce and this might be okay. I ended up draining all of the wasted butter/juice and frying it in a skillet to get some crisp on it. Obviously it got overcooked at that point but that was the only thing I knew to do to salvage it. I cannot recommend this recipe but if you do choose to make it, start with half of the sauce and go from there.'